In [1]:
import pyaudio
import numpy

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(f'{i} ' + p.get_device_info_by_index(i).get('name'))

0 Microsoft Sound Mapper - Input
1 Microfone Externo (Realtek(R) A
2 Microfone (Realtek(R) Audio)
3 Microsoft Sound Mapper - Output
4 Fones de ouvido (Realtek(R) Aud
5 Alto-falantes (Realtek(R) Audio
6 Driver de captura de som primário
7 Microfone Externo (Realtek(R) Audio)
8 Microfone (Realtek(R) Audio)
9 Driver de som primário
10 Fones de ouvido (Realtek(R) Audio)
11 Alto-falantes (Realtek(R) Audio)
12 Alto-falantes (Realtek(R) Audio)
13 Fones de ouvido (Realtek(R) Audio)
14 Microfone (Realtek(R) Audio)
15 Microfone Externo (Realtek(R) Audio)
16 Microfone (Realtek HD Audio Mic input)
17 Mic in at front panel (black) (Mic in at front panel (black))
18 Headphones (Realtek HD Audio 2nd output)
19 Speakers 1 (Realtek HD Audio output with HAP)
20 Speakers 2 (Realtek HD Audio output with HAP)
21 Alto-falante (Realtek HD Audio output with HAP)
22 Mixagem estéreo (Realtek HD Audio Stereo input)


Os dois testes retornam valores que não variam muito e são extremamente baixos.

In [ ]:
import sounddevice as sd
from numpy import linalg as LA

fs = 44100

def point(seconds):
    myrecording = sd.rec(int(seconds * fs), samplerate= fs, channels= 1)
    sd.wait()
    max = numpy.max(myrecording)
    print(max)

duration = 10

def print_sound(indata, outdata, frames, time, status):
    volume_norm = numpy.linalg.norm(indata) * 10
    print(int(volume_norm))

with sd.Stream(callback=print_sound):
    sd.sleep(duration * 1000)

Nesse teste tentei usa primeiro o método encontrado no link: https://elec-club-iitb.github.io/blog/2016/08/sound-meter/  
Mas os valores retornados são muito altos e não estão no formato dbs.  
Usei o código da fonte anterior com a fonte encontrada no seguinte link: https://github.com/SuperShinyEyes/spl-meter-with-RPi  
Depois de muitas tentativas, o algoritimo funciona mas não retorna absolutamente nada além de '+info' ou '-info'.
Finalmente consegui fazer funcionar, e não só com a Webcam mas também com o microfone do fone de celular.

In [54]:
import time, audioop
from scipy.signal import lfilter, bilinear

def A_weighting(fs):
    f1 = 20.598997
    f2 = 107.65265
    f3 = 737.86223
    f4 = 12194.217
    A1000 = 1.9997

    NUMs = [ (2 * numpy.pi * f4) ** 2 * (10 ** (A1000 / 20)), 0, 0, 0, 0 ]
    DENs = numpy.polymul( [1, 4* numpy.pi * f4, (2 * numpy.pi * f4) ** 2],
                        [1, 4 * numpy.pi * f1, (2 * numpy.pi * f1) ** 2] )
    DENs = numpy.polymul( numpy.polymul(DENs, [1, 2 * numpy.pi * f3]),
                        [1, 2 * numpy.pi * f2] )

    return bilinear(NUMs, DENs, fs)

def rms_flat(a):
    return numpy.sqrt(numpy.mean(numpy.absolute(a) ** 2))




CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECOND = 5

NUMERATOR, DENOMINATOR = A_weighting(RATE)

stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK, input_device_index=2)

def is_meaningful(old, new):
    return abs(old - new) > 3

def listen(old=0, error_count=0, min_decibel=100, max_decibel=0):
    print('Listening...')
    while True:
        try:
            block = stream.read(CHUNK)
        except IOError as e:
            error_count += 1
            print(" (%d) Error recording: %s" % (error_count, e))
        else:
            decoded_block = numpy.fromstring(block, numpy.int16)

            y = lfilter(NUMERATOR, DENOMINATOR, decoded_block)
            new_decibel = 20 * numpy.log10(rms_flat(y))
            if is_meaningful(old, new_decibel):
                old = new_decibel
                print('A-weighted: {:+.2f} dB'.format(new_decibel))
            
listen()

Listening...
A-weighted: +28.89 dB
A-weighted: +22.66 dB
A-weighted: +32.38 dB
A-weighted: +28.74 dB


C:\Users\Arthur Cezar\AppData\Local\Temp\ipykernel_12608\2726372211.py:47: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  decoded_block = numpy.fromstring(block, numpy.int16)


A-weighted: +25.25 dB
A-weighted: +18.10 dB
A-weighted: +21.81 dB
A-weighted: +26.15 dB
A-weighted: +31.02 dB
A-weighted: +24.48 dB
A-weighted: +27.87 dB
A-weighted: +19.34 dB
A-weighted: +24.68 dB
A-weighted: +20.09 dB
A-weighted: +13.69 dB
A-weighted: +22.82 dB
A-weighted: +26.35 dB
A-weighted: +13.38 dB
A-weighted: +16.49 dB
A-weighted: +12.00 dB
A-weighted: +8.38 dB
A-weighted: +4.55 dB
A-weighted: +7.57 dB
A-weighted: +4.42 dB
A-weighted: +1.38 dB
A-weighted: -2.41 dB
A-weighted: +3.42 dB
A-weighted: -0.99 dB
A-weighted: +3.39 dB
A-weighted: -0.55 dB
A-weighted: +4.39 dB
A-weighted: +0.00 dB
A-weighted: +10.96 dB
A-weighted: +1.24 dB
A-weighted: +6.54 dB
A-weighted: -3.30 dB
A-weighted: +2.54 dB
A-weighted: -0.59 dB
A-weighted: +6.07 dB
A-weighted: -0.38 dB
A-weighted: +6.92 dB
A-weighted: -2.73 dB
A-weighted: +0.44 dB
A-weighted: -4.03 dB
A-weighted: -0.52 dB
A-weighted: +3.72 dB
A-weighted: -0.88 dB
A-weighted: +2.71 dB
A-weighted: -1.12 dB
A-weighted: -4.56 dB
A-weighted: +3.52

KeyboardInterrupt: 

Até funciona, mas não do jeito que eu gostaria...  
Talvez tentar mesclar com alguma outra solução.

In [49]:
for i in range(0, 10):
    data = stream.read(CHUNK)

    reading = audioop.max(data, 2)
    print(reading)
    time.sleep(1)

2
3
9
7
5
4
4
5
5
3
